# Introduction 

With the data at disposition of different areas of the city, the goal of the project is to analyze the traffic of Nice area in the year 2019 and 2020. The aim is to identify different seasonal and weekly paths of the traffic over the periods considered, and try to study the impact of the restrictions due to Covid2019 in the traffic dynamic. The data are used with respect their temporal order, daily time series are created with respect to the variables at disposition. In the first section the techniques and algorithms used are presented with a theoretical approach. In the second section Promenade Des Anglais data are considered, after having explain the pre-process implemented result are showed for both 2019 and 2020.    

# Methodology 

In order to identify seasonal and weekly traffic paths, using daily time series, clustering procedures are proposed. Clustering is the task of dividing the time series data into a number of groups such that data in the same groups are more similar, present analogous behaviours during the day, than those in other groups. To better identify the path represented by a group (cluster) a barycenter, also call centroid, is computed. Since I am dealing with temporal data I have to define a strategy in order to compare different series both for assign each series in a cluster and update the centroid. I would need a dissimilarity measure that is able to “match” a point of time series $x$ even with “surrounding” points of time series $y$. The Euclidean distance, assumes the $i-th$ point of the $x$ series is aligned with the $i-th$ point of the $y$ series, it will produce a pessimistic dissimilarity measure. Two clustering procedures are discussed below: Soft-Dynamic Time Warping with K-means algorithm and Global Alignment kernel (GAK) inside of a kernel K-means algorithm. The two procedure are connected each other and can be used together as showed later. 

In addition to futher highlight long-term trends in the traffic behaviour a simple moving average calculation is taken in account. This techniques allows to analyze temporal data by creating a series of averages of different subsets of the full data set that delete short-term fluctuations.

## Soft-Dynamic Time Warping and K-means algorithm. 

Dynamic Time Warping is a technique to measure similarity between two temporal sequences considering not only the temporal alignment but every binary alignment of the two series. For example a similar traffic condition based on different variables could be recognized in different hours of the day in two different series. The calculation of the DTW similarity involves a dynamic programming algorithm that tries to find the optimum warping path between two series under certain constraints.

Given two multivariate series that corresponds to two different days:

$x \in R^{d \times n}$ and $y \in R^{d \times n}$ valued in $R^d$ ($d$ dimension of the multivariate time series). 

Consider a function in order to compare different points of the two series $(x_i \in R^{d}$ and $y_j \in R^{d}) \ \ d \ \ : R^d × R^d \Rightarrow R$, such as $d(x_i, y_j)= (\sum_{i,j=1}^n{(|x_i - y_j|^p})$, where usually $p=2$ and $d(x_i, y_j)$ is the quadratic Euclidean distance between two vectors. 

A matrix of similarity is computed: 

$\Delta(x,y) := [d(x_i,y_j)]_{i,j} \in R^{n \times n}$

$\Delta(x,y)$ can also be defined as local cost matrix, such a matrix must be created for every pair of series compared.  

The DTW algorithm finds the path that minimizes the alignment between $x$ and $y$ by iteratively stepping through  $\Delta(x,y)$, starting at $[d(x_i,y_j)]_{1,1}$ (bottom left) and finishing at $[d(x_i,y_j)]_{n,n}$ (upper right).  and aggregating the cost \footnote{Sardà-Espinosa:"Comparing Time-Series Clustering Algorithms in R Using the dtwclust Package" chapter 2}. At each step, the algorithm finds the direction in which the cost increases the least allowing 3 elementary moves $\rightarrow$, $\uparrow$, $\nearrow$ . To improve the discrimination between different warped paths a chosen constraint can be specyfied. This constraint typically consists in forcing paths to lie close to the diagonal of the local cost matrix \footnote{Sakoe,Chiba: “Dynamic programming algorithm optimization for spoken word recognition,” IEEE Transactions on Acoustics, Speech and Signal Processing, vol. 26(1), pp. 43–49}. 

By considering $A_{n,n} \subset \{0, 1\}^{n,n}$ the set of all binary alignment between two time series $x$ and $y$ of the same lenght $n$, the DTW similarity measure reads as follow: 


\begin{equation}
DTW(x,y)= \min_{A \in A_{n,n} } \langle\,A,\Delta(x,y) \rangle
\end{equation}

This creates a warped “path” between $x$ and $y$ that aligns each point in $x$ to the nearest point in $y$. 

However I can not define dynamic time warping as a distance because does not satisfy the triangular inequality,
moreover it is not differentiable everywhere due to the  $\min$ operator.  

Soft-Dynamic Time Warping is a variant of DTW that is differentiable. It uses the log-sum-exp formulation \footnote{ Cuturi,Blondel:"Soft-DTW: a Differentiable Loss Function for Time-Series"}:

\begin{equation}
DTW^{\gamma}(x,y)= - \gamma \log \sum_{A \in A_{n,n}} exp(- \frac{\langle\,A,\Delta(x,y) \rangle}{\gamma})
  \ \ where \ \ \gamma  \geq 0
\end{equation}

  

Despite considering all alignments and not just the optimal one, soft-DTW can be computed in quadratic time $O(d \times n^2)$ as DTW, however as DTW soft-DTW does not satisfy the triangular inequality. Soft-DTW is a symmetric similarity measure, it supports multivariate series as DTW, and it can provide differently smoothed results by means of a user-defined parameter $\gamma$. 






The "path" created between  $x$ and $y$ is smoother than the one created with DTW. Soft-DTW depends on a hyper-parameter $\gamma$ that controls the smoothing. As showed in equation (3) DTW corresponds to the limit case when $\gamma$ =0. 


\begin{equation}
DTW^{\gamma}(x,y)=\begin{cases}
                     \min_{A \in A_{n,n} } \langle\,A,\Delta(x,y) \rangle ,\ \  \gamma=0 \\
                     - \gamma \log \sum_{A \in A_{n,n}} exp(- \frac{\langle\,A,\Delta(x,y) \rangle}{\gamma}), \ \ \gamma \geq 0 \\
                     \end{cases}
\end{equation}




SoftDTW is then using with Centroid-based clustering, such as K-means and K-medoids. These algorithms use an iterative way to create the clusters by moving data points from one cluster to another, based on a distance measure, starting from an initial partitioning. \footnote{K-medoids clustering is very similar to the K-means clustering algorithm. The major difference between them is that while a cluster is represented with its center in the K-means algorithm, it is represented with the most centrally located data point in a cluster in the K-medoids clustering}.


The soft-DTW is used in K-means algorithm to assign the series to the clusters and to upload the centroid of the cluster. Centroid in a cluster correspond to the multivariate time series $\in R^{d \times n }$ that minimizes the sum of the similarity measures between that time series and all time series inside the cluster. The centroid in a cluster is computed artificially, it does not correspond to a time series inside the cluster. Given the $ts$ multivariate time series each of them composed by $n$ daily observations the algorithm work as follow: 


$$
\begin{cases}
\textbf{Algorithm} \ \ k-meansclustering \ \ (T,K) \\
\ \ \ \ Input: T = (t_1, t_2, ..., t_{ts}) \ \ set \ \ of \ \ daily \ \ time \ \ series \ \ where \ \ the  \ \ generic \ \ series \ \ t_i\in R^{d \times n } \\
 \ \ \ \ Input: k \ \ the \ \ number \ \ of \ \ clusters \\
 \ \ \ \ Output: {c_1, c_2, ..., c_k} \ \ (partion\ \ of\ \ time\ \ series \ \ in \ \ cluster, \ \ centroid\ \ of \ \ each\ \ cluster \ \ c_i\in R^{d \times n }) \\
\ \ \ \ p=0 \\
 \ \ \ \ Randomly \ \ choose \ \ k \ \ series\in R^{d \times n } \ \ and \ \ make \ \ them \ \ as \  initial \ \ centroids \ \ ({c_1^{(0)}, c_2^{(0)}, ..., c_k^{(0)}}) \\ 
 \ \ \ \  \textbf{repeat} \\
  \ \ \ \ \ \ \ \  Assign \ \ each \ \ time \ \ series \ \ to \ \ the \ \ nearest \ \ cluster \ \ using \ \ \arg\min_k DTW^{\gamma}(c_k^{(p)},t_i) \\
  \ \ \ \ \ \ \ \ p=p+1 \\
   \ \ \ \ \ \ \ \ // \ \  \textbf{Centroid update} \\
   \ \ \ \ \ \ \ \ \textbf{for} \ \ j=1 \ \ to \ \ k \ \ \textbf{do} \\
    \ \ \ \ \ \ \ \ \ \ Update \ \ the \ \ centroid \ \ c_j^{(p)} \ \ by  \ \ considering \ \ the  \ \ subset \ \ of\ \ series\ \ (t_1, t_2, ..., t_{t} \ \ where  \ \ t \leq ts) \ \ that \ \ belong \ \ to \ \ cluster \ \ j  \ \ c_j^{(p)}= \arg\min_j\sum_{i=1}^{t}DTW^{\gamma}(c_j,t_i)    \\
    \ \ \ \ \ \ \ \ \textbf{end for} \\ 
     \ \ \ \  \textbf{until} \\ \ \  c_j^{(p)} \approx  c_j^{(p-1)} \ \  \ \ j = 1, 2, ..., k \\
     \ \ \ \  Return \ \ c_1 ,c_2 ,... ,c_k \\
\end{cases}
$$


Sometimes different initializations of the centroids lead to very different final clustering results. To overcome this problem the K-means algorithm is run 5 times  with different centroids randomly placed at different initial positions. The final results will be the best output of the 5 times consecutive runs in terms of inertia.

## Global Alignment Kernel and Kernel K-Means 

Kernel methods use a kernel function in order to separate high dimensional data. Given two time series $x$,$y \in R^{d \times n }$ A kernel $k(,)$ is defined as 
\begin{equation}
k(x,y)=  \langle\,\rho (x) ,\rho (y) \rangle_{\nu}
\end{equation}
where an appropriate non linear mapping $\rho: R^{d \times n } \rightarrow \nu$ is used to compute  the dissimilarity measure $k(x,y)$ in some (unknown) embedding space $\nu$ (also know as latent space).  This approach is called the "kernel trick". It is used to map time series in a higher-dimensional feature space  $\nu$ by computing the inner products between the images of pairs of data using a nonlinear function $\rho$. The "kernel trick" perform computations in the embedding space $\nu$ by the inner product of the transformed vectors $\rho (x) ,\rho (y)$ in the higher dimensional space. For example, one can compute distance between $x$ and $y$ in $\nu$ as
\begin{equation}
\begin{split}\| \rho(\mathbf{x}) - \rho(\mathbf{y})\|_\mathcal{\nu}^2
    &= \left\langle \rho(\mathbf{x}) - \rho(\mathbf{y}),
                    \rho(\mathbf{x}) - \rho(\mathbf{y})
       \right\rangle_{\mathcal{\nu}} \\
    &= \left\langle \rho(\mathbf{x}), \rho(\mathbf{x})
       \right\rangle_{\mathcal{\nu}}  +
       \left\langle \rho(\mathbf{y}), \rho(\mathbf{y})
       \right\rangle_{\mathcal{\nu}}  - 2
       \left\langle \rho(\mathbf{x}), \rho(\mathbf{y})
       \right\rangle_{\mathcal{\nu}} \\
    &= k(\mathbf{x}, \mathbf{x}) + k(\mathbf{y}, \mathbf{y})
       - 2 k(\mathbf{x}, \mathbf{y})\end{split}
\end{equation}
such computations are used in the Kernel K-means algorithm as illustrated later  \footnote {Dhillon,Guan,Kulis :“Kernel k-means, Spectral Clustering and Normalized Cuts”}.
In Kernel K means before clustering, time series are mapped to a higher-dimensional feature space $\nu$ using a nonlinear function$\rho$, and then kernel K-means partitions the series by linear separators in the new space.
Kernel methods require only a user-specified kernel $k(,)$, such  as similarity function over pairs of data in raw representation. 

The Global Alignment Kernel (GAK) is a kernel that operates on time series. Considering two time series $x$,$y \in R^{d \times n }$ I indicate with $\pi$ a generic warping function, composed by a pair of integral vectors: $\pi_1$ for $x$ and $\pi_2$ for $y$. $\pi$ maps the generic alignment between two series from $\Delta(x,y)_{1,1}$ (bottom left) to $\Delta(x,y) _{n,n}$ (upper right) by allowing 3 elementary moves $\rightarrow$, $\uparrow$, $\nearrow$ as previoulsy showed. The (GAK) for a given bandwith $\sigma$ is defined as:
\begin{equation}
k_{GAK}^{\gamma}(x,y) = \sum_{\pi \in \mathcal{A}(n,n)}\prod_{i=1}^{ | \pi | }
            \exp \left( - \frac{ \left\| x_{\pi_1(i)} - y_{\pi_2{i}} \right\|^2}{2 \sigma^2} \right)
\end{equation}
where $| \pi |$ is the length of the generic alignment and the discrepancy between any two points $x_i$ and $y_i$ observed in $x$ and $y$ is the squared euclidean distance. 

As reported in \footnote {Cuturi:"Fast Global Alignment Kernels"} the bandwidth $\sigma$ can be set as a multiple of a simple estimate of the median distance of different points observed in different time-series of the training set, scaled by the square root of the length of time-series in the training set (In my case all daily multivariate time series have the same lenght previously indicated with $n$). The suggested estimation is 
\begin{equation}
\sigma = median||x - y||\sqrt{n}
\end{equation}
and it is available in $\textbf{tslearn}$ \footnote{Tavernard et.Al: "Tslearn, A Machine Learning Toolkit for Time Series Data"}. 

The(GAK) is related to softDTW, it can be defined as the exponentiated soft-minimum of all aligment distances:
\begin{equation}
k_{GAK}^{\gamma}(x,y)=\sum_{A \in A_{n,n}} exp(- \frac{\langle\,A,\Delta(x,y) \rangle}{\gamma}) 
\end{equation}
where the $\gamma$ hyperparameter that controls the smoothness in softDTW is linked with $\sigma$ by
\begin{equation}
\gamma = 2 \sigma^2
\end{equation}

Through this relation GAK can be used to estimate $\gamma$ hyperparameter of softDTW. 

Both GAK and softDTW, that scales in $O(d \times n^2)$, incorporates the set of all possible alignments $A \in A_{n,n}$ in the cost matrix $\Delta(x,y)$. For this reason they provide richer statistics than the minimum of that set, which is instead the unique quantities considered by DTW. However only Global Alignment Kernel (GAK) is positive definite \footnote{Blondel,Mensch,Vert:"Differentiable Divergences Between Time Series"}.  

Given a kernel $k_{GAK}^{\gamma}(,)$ and as input a set of daily time series $T=(t_1,t_2,.....,t_{ts})$ where the generic daily series $t_i \in R^{d \times n }$, the $ts  \times ts$ matrix
 $K= (k_{GAK}^{\gamma}(t_i , t_j))_{ij}$
is called the Kernel matrix. Furthermore, the positive definiteness of kernel function translates $k_{GAK}^{\gamma}(,)$ in practice into the positive definiteness of the so called Kernel matrix $K$ \footnote{Cuturi:"Positive Definite Kernels in Machine Learning" pag 9}. .

Positive definite Kernel Matrix can be used in kernel K-means algorithm. 

The aim of kernel Kmeans algorithm is to partition the set of daily time series $T=(t_1,t_2,.....,t_{ts})$ in $k$ clusters denoted as $\{C_j \}_{j=1}^k$. The algorithm introduce also weight for each time series denoted by $w$. 
A first significant difference (when compared to k-means) is that clusters centers are never computed explicitly, hence time series assignments to cluster are the only kind of information available once the clustering is performed. The algorithm only finds the "best" cluster representative in the embedding space. For the generic cluster $j$ the "best" cluster representative is computed as: 
\begin{equation}
m_j=\frac{\sum_{t_i \in C_j}w(t_i)\rho(t_i)}{\sum_{t_i \in C_j}w(t_i)}
\end{equation}

The squared euclidean distance from $\rho(t_i)$ to $m_j$ used to update the partitioning of the time series at every step read as: 
\begin{equation}
\begin{split}\| \rho(t_i) - m_j\|_\mathcal{\nu}^2
    &= \left\langle \rho(t_i) - m_j,
                    \rho(t_i) - m_j
       \right\rangle_{\mathcal{\nu}} \\
    &= \left\langle \rho(t_i), \rho(t_i)
       \right\rangle_{\mathcal{\nu}}  +
       \left\langle m_j,m_j
       \right\rangle_{\mathcal{\nu}}  - 2
       \left\langle \rho(t_i), m_j
       \right\rangle_{\mathcal{\nu}} \\
\end{split}    
\end{equation}
where the inner product of time series are computed using kernel function $k_{GAK}^{\gamma}(,)$ and contained in the kernel matrix $K$

The kernel K-means algorithm finally reads as follow: 


$$
\begin{cases}
\textbf{Algorithm} \ \  Kernel k-meansclustering \ \ (T,K) \\
\ \ \ \ Input: K  \ \ Kernel \ \ Matrix  \\
 \ \ \ \ Input: k \ \ the \ \ number \ \ of \ \ clusters \\
 \ \ \ \ Output: {C_1, C_2, ..., C_k} \ \ (partion\ \ of\ \ time\ \ series \ \ in \ \ clusters) \\
\ \ \ \ p=0 \\
 \ \ \ \ Randomly \ \ assign\ \ each \ \ series \ \ to \ \ a \ \ cluster \ \ ({C_1^{(0)}, C_2^{(0)}, ..., C_k^{(0)}}) \\ 
 \ \ \ \  \textbf{repeat} \\
  \ \ \ \ \ \ \ \  For  \ \ every \ \ cluster \ \ \{C_j \}_{j=1}^k \ \ find \ \ the \ \ "best" \ \ cluster \ \ representative \ \ m_j=\arg\min_m \sum_{t_i \in C_j}w(t_i)|| \rho(t_i) - m\|_\mathcal{\nu}^2 \\
   \ \ \ \ \ \ \ \  Update  \ \ the \ \ partitioning \ \ of \ \ time \ \ series \ \ by \ \ considering \ \ \ \ the \ \ "best" \ \ cluster \ \ representative \ \ previously \ \ computed \ \ J^*(t_i)=\arg\min_j || \rho(t_i) - m\|_\mathcal{\nu}^2 \\
  \ \ \ \ \ \ \ \ p=p+1 \\
     \ \ \ \  \textbf{until} \\ \ \  C_j^{(p)} \approx  C_j^{(p-1)} \ \  \ \ j = 1, 2, ..., k \\
     \ \ \ \  Return \ \ c_1 ,c_2 ,... ,c_k \\
\end{cases}
$$



A Second difference with respect to k-means is that clusters generated by kernel k-means are phase 
dependent rather than in shape. For instance two days that present same level of traffic congestion (same shape) reached in different hours (different phase) can be assigned to different clusters. This is because Global Alignment Kernel is not invariant to time shifts, as demonstrated in \footnote{Janati, Cuturi, Gramfort: “Spatio-Temporal Alignments: Optimal transport through space and time,”} for the closely related soft-DTW, however the difference in time shift becomes less emphatic when considering series of the same lenght. 

As mentioned before GAK and softDTW are closely related and can be used together. GAK can be used to tune the hyperparameter $\gamma$ in softDTW by applying equation() and equation() while the latter can be used to computed the cluster center once kernel k-means is computed. The barycenter corresponds to the time series that minimizes the sum of the distances between that time series (computed artificially) and all the time series in the cluster. 
Given the set of series  $(t_1, t_2, ..., t_{t}$ that belong to cluster $j$ the centroid $c_j\in R^{d \times n}$ reads as:
\begin{equation}
c_j=\arg\min_j\sum_{i=1}^{t}DTW^{\gamma}(c_j,t_i) 
\end{equation}




## Simple moving average

The moving average is commonly used with time series to smooth random short-term variations and to highlight other components such as trend present in the data. This technique is used to highlight the traffic trend of long periods (see most trafficated months), deleting the impact of the traffic peak in the morning and in afternoon. The simple moving average is the unweighted mean of the previous $M$ data points. The selection of $M$ (sliding window) depends on the amount of smoothing desired. By increasing the value of $M$ improves the smoothing at the expense of accuracy. For a sequence of values, the simple moving average at time period $t$ reads as follows:

\begin{equation}
 SMA_t=\frac{x_t+x_{t-1}+x_{t-2}+.....+x_{M-(t-1)}}{M}
\end{equation}




# Promenade Des Anglais

The data at disposition are collected from 9 detectors positioned on the Promenade in both directions. 3 detectors collect the traffic in the north direction and 6 detectors collect the traffic in the south direction (facing the sea). Two period of time are registered from 01/06/2019 to 31/12/2019 and from 06/02/2020 to 30/12/2020. Two variables are reported for each lane of the road: the flow (n° veh/h) indicated as $q(x,t)$ and the occupancy rate of the detector in percentage indicated as $o(x,t)$. The data are recorded every minute. Rough data needs to be treated and aggregated. Here a list of operation computed in order to study the dynamics in both intervals time 2019 and 2020:

1.	 Duplicate removal.

2.	Mark as missing observations of the occupancy rates that are greater than 50%.

3.	Mark as missing observations of the flow that are greater than 60 veh/h and have an occupancy rate equal to 100%.

4.	For every detector all lanes are aggregated together (any lane containing NaN, cause the resulting output column flow or occupancy to be NaN as well).

5.	By looking at the fundamental diagram outlier observations are removed. 

6.	Fill jumps in the series due to the omission of data with Nans values for both flow and occupancy rate. 1430 omissions in 2019 period and 2401 omissions in 2020 period.   

7.	6-minute averages aggregation to mitigate the impact of traffic lights.

8.	Compute the percentage of missing values in the series in order to decide which detectors can be used in the analysis. 

9.	Fill Nans values of usable detector with an imputation procedure. 
 
To apply the clustering procedure the NaNs values must be filled. The imputation procedure takes in account the temporal order of observations, thus if there are too many consecutive NaNs to fill the procedure is not applied. 
The procedure reads as follow: 

1.	If an isolated Nan value is recognized (both previous and next observations are present), the observation before the Nan value is propagate forward to fill the missing value.

2.	If the number of consecutive Nan values is lower than 20 (2 hours). The consecutive Nans are filled with a linear method based on the temporal alignment of the previous observations available. Otherwise if the number of consecutive Nans exceed 2 hours the imputation is not performed.


Time series, flow and occupancy rate, are preprocessed using normalization over all periods (51360 observations for 
2019 and 66480 observations for 2020  6-minute averages). This scaler is such that each output time series is in the range [0,1] allowing to have identical scales for time series with originally different scales ($veh/h$ and $ \%$): 
$$q_{norm}(t,x)=\frac{q(t,x)-MIN}{MAX-MIN}$$. 
$$o_{norm}(t,x)=\frac{o(t,x)-MIN}{MAX-MIN}$$. 

The MinMaxScaler function from the Python machine learning library  $\textbf{Scikit-learn}$ \footnote{Pedregosa et.Al "Scikit-learn: Machine Learning in Python"}  trasforms each values of the time series proportionally within the range [0,1] preserving the shape. Density and flow scaled series are not amplitude invariant, they do not have the same standard deviation (reached instead by using standardization). 

The inverse_transform method of MinMaxScaler, that undo the scaling of a data point  according to feature_range,  $$q(t,x)=q(t,x)_{norm}*(MAX-MIN)+ MIN$$  $$o(t,x)=o(t,x)_{norm}*(MAX-MIN)+ MIN$$ is used in the centroids representations to have a better comprehension of paths, no more in the [0,1] range.

Despite the $MIN \ \ MAX$ for $q(t,x)$ and $o(t,x)$ are defined over the years 2019 and 2020, the clustering procedure is applied to the $ts$ daily multivariate time series with $n$ daily observations (where $n$ is equal to 240 observations with 6-minute averages). The choice of scaling the two varaibles with respect to the entire periods not with respect to the single daily time series is done to preserve variability with respect to different days of the week. For example I simple assume that the $MAX$ flow reached on Sunday or Saturday is really low compared to $MAX$ flow reached on another working day. 

To create the $ts$ daily normalized multivariate time series a simple $\textbf{reshape}$ procedure is applied: 

$[ts*n,2] \implies [ts,n,d]$ where $d=2$ is the dimensionality of the daily  multivariate time series (flow and occupancy rate).


## 2019 

For 2019 period only 3 detectors contain all data from 1/6 to 31/12. A list of the main problem reported for the other detectors is available in the Appendix. The three detector used in the analysis are C601,C009, C094. The locations are illustrated in Figure 2.1 and Figure 2.2.  

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{terminal1.png}
    \caption{}
    \label{Fig 2.1}
\end{figure}
\emph{\small Figure 2.1 Location of C601 detector}



\begin{figure}
    \centering
    \includegraphics[scale=0.5]{gambetta cronstadt.png}
    \caption{}
    \label{Fig 2.2}
\end{figure}
\emph{\small Figure 2.2 Location of C009 and C094 detector}


### C601

To have an idea of the dynamic of the traffic over all period available the simple moving average with a window size of one week is computed for C601 detectors located near the airport, terminal 1. As showed in Figure 3.1 the traffic in term of $q(t,x)$ (number of veh/h) present a seasonal behaviours. I can distinguish three different level of traffic also detected by the kernel K-means algorithm in Figure 3.2 . The most trafficated periods are from the start of June until the second week of July and from the start of September to the third week of October. The less trafficated period is the summer that coincide roughly with the school break: from 5/7 to 2/9. The winter period from the third week of October to the third week of December( before Christmas Holidays) as a low level of traffic compared with the September period.      

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{SMA C601 2019.png}
    \caption{}
    \label{Fig 3.1}
\end{figure}
\emph{\small Figure 3.1  Simple moving average 2019 period }


In Figure 3.2 the Global Alignment Kernel is seeded in the kernel K-Means algorithm with four clusters. 214 bi-variate (flow and occupancy rate) daily time series are taken as input. The Algorithm in addition to the three seasonal behaviours listed above recognize also the no-working days (Saturday, Sunday and Public Holidays). 

\begin{figure}
    \centering
    \includegraphics{GAK C601 2019 K=4.png}
    \caption{}
    \label{Fig 3.2}
\end{figure}
\emph{\small Figure 3.2 Kernel K-means, four clusters 2019 period}

In Figure 3.3 a random sample of bi-variate time series for each cluster is represented. To better render each cluster, after all time series are assigned with Kernel K-Means, the centroids of the clusters are computed using softDTW equation().The $\gamma$ parameter is set to 41.5936. 
The first cluster $k=0$ identifies the summer in which the peak of the morning is flatter than the ones present in cluster $k=1$ and $k=3$ for both flow and occupancy rate. The second cluster $k=1$ identifies the winter behaviour, where the occupancy rate in the range 10:00 - 16:00 is lower than the ones present in cluster $k=3$. The third cluster $k=2$ portrays the dynamic of the traffic during no-working days, where during the morning hours 7:00 - 10:00 the traffic level remains low. The fourth cluster $k=3$ describes the most trafficated period in which even outside traffic peaks in the morning and in the afternoon the occupancy rate register higher values with respect to the other clusters.  

\begin{figure}
    \centering
    \includegraphics[scale=0.7]{GAK centroids K=4 2019.png}
    \caption{}
    \label{Fig 3.3}
\end{figure}
\emph{\small Figure 3.3 Centroids of Kernel K-means, four clusters 2019 period, computed with softDTW}

In Figure 3.4 to better distinguish the behaviour of the first $k=0$ and fourth $k=3$ cluster weekly series of flow (veh/h) of August and September are compared. The September series presents a higher level of traffic especially in the first part of the day.   

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{GAK august vs september 2019.png}
    \caption{}
    \label{Fig 3.4}
\end{figure}
\emph{\small Figure 3.4 Comparison of working days in a week of August and September 2019 }

In Figure 3.5 only the subperiod from 2/9 to 21/12 is considered. The Global Alignment Kernel is seeded in the kernel K-Means algorithm with four clusters, and 111 bi-variate (flow and occupancy rate) daily time series are taken as input. Despite some weekend days form a unique cluster, the seasonal behaviour is once again confirmed with a separation between September and the winter period starting from the third week of October. To separate better these two behaviour in Figure  3.5.2 weekly series of occupancy rate of September and November are compared. As already mention the occupancy rate in the interval 10:00 - 16:00 is the main difference between the two period.    

\begin{figure}
    \centering
    \includegraphics{GAK K=4 winter 2019.png}
    \caption{}
    \label{Fig 3.5}
\end{figure}
\emph{\small Figure 3.5 Kernel K-means, four cluster from 2/9/2019 to 21/12/2019}

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{C601 setember vs november 2019.png}
    \caption{}
    \label{Fig 3.5.2}
\end{figure}
\emph{\small Figure 3.5.2 Comparison of working days in a week of September and November 2019}

### C009/C094

In this section results for C009/C094 are showed. C009 and C094 are used in this section without any distinction, due to their proximity. The algorithm and the moving average indicate a different behaviour of the traffic especially in the summer months compared with the C601. This mainly due to the location of the detectors close to the city centre of Nice and the beach as illustrated in Figure 2.2. In particular comparing Figure 3.5.5 with 3.1, in which the moving average of one week for C009 and C601 detector are computed, the month of August in Figure 3.5.5 does not present an inflection in terms of flow in the two centrals weeks.      

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{SMA C009.png}
    \caption{}
    \label{Fig 3.5.5}
\end{figure}
\emph{\small Figure 3.5.5 Simple moving average 2019 period}

In Figure 3.6 the softDTW is seeded in the K-Means algorithm with four clusters. The $\gamma$ parameter estimated using equation () and equation () is set to 18.5761. Again 214 bi-variate (flow and occupancy rate) daily time series are taken as input. The Algorithm put together the month of August and Saturdays, while Sundays form an unique cluster. The remaining days of the week are represent by the other two clusters which mostly differ for the traffic dynamic in the range 10:00 - 16:00 and in the range 20:00 - 24:00. in addition to the three seasonal behaviours listed above recognize also the no-working days (Saturday, Sunday and Public Holidays). 

\begin{figure}
    \centering
    \includegraphics{softDTW C009 K=4 2019.png}
    \caption{}
    \label{Fig 3.6}
\end{figure}
\emph{\small Figure 3.6 K-means, four clusters 2019 period }

\begin{figure}
    \centering
    \includegraphics{softDTW centroids K=4 2019.png}
    \caption{}
    \label{Fig 3.7}
\end{figure}
\emph{\small Figure 3.7 Centroids of K-means, four clusters 2019 period}

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{softDTW july vs november 2019.png}
    \caption{}
    \label{Fig 3.8}
\end{figure}
\emph{\small Figure 3.8 Comparison of working days in a week of July and November 2019}

\begin{figure}
    \centering
    \includegraphics{softDTW K=4 winter 2019.png}
    \caption{}
    \label{Fig 3.9}
\end{figure}
\emph{\small Figure 3.9 K-means,four cluster from 2/9/2019 to 21/12/2019}

\begin{figure}
    \centering
    \includegraphics{softDTW centroids K=4 winter 2019.png}
    \caption{}
    \label{Fig 3.10}
\end{figure}
\emph{\small Figure 3.10 Centroids of K-means, four clusters from 2/9/2019 to 21/12/2019}

## 2020 

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{SMA C601 2020.png}
    \caption{}
    \label{Fig 3.11}
\end{figure}
\emph{\small Figure 3.11 Simple moving average 2020 period}

\begin{figure}
    \centering
    \includegraphics{softDTW K=5 2020.png}
    \caption{}
    \label{Fig 3.12}
\end{figure}
\emph{\small Figure 3.12 K-means, five clusters 2020 period}

\begin{figure}
    \centering
    \includegraphics{softDTW centroids K=5 2020.png}
    \caption{}
    \label{Fig 3.13}
\end{figure}
\emph{\small Figure 3.13 Centroids of K-means, five clusters 2020 period}

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{C601 april vs may 2020.png}
    \caption{}
    \label{Fig 3.14}
\end{figure}
\emph{\small Figure 3.14 Comparison of working days in a week of April and May 2020}

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{C601 august vs september .png}
    \caption{}
    \label{Fig 3.15}
\end{figure}
\emph{\small Figure 3.15 Comparison of working days in a week of August and September 2020}

\begin{figure}
    \centering
    \includegraphics{GAK flow midterm 2020.png}
    \caption{}
    \label{Fig 3.16}
\end{figure}
\emph{\small Figure 3.16 kernel K-means on flow of all detectors, six clusters from 1/9/2020 to 17/10/2020}

\begin{figure}
    \centering
    \includegraphics{GAK occupancy midterm 2020.png}
    \caption{}
    \label{Fig 3.17}
\end{figure}
\emph{\small Figure 3.17 kernel K-means on occupancy rates of all detectors, six clusters from 1/9/2020 to 17/10/2020}

## 2019 vs 2020

\begin{figure}
    \centering
    \includegraphics[scale=0.5]{FD C601 2019.png}
    \caption{}
    \label{Fig 3.18}
\end{figure}
\emph{\small Figure 3.18 Fundamental diagram 2019}


\begin{figure}
    \centering
    \includegraphics[scale=0.5]{FD C601 2020.png}
    \caption{}
    \label{Fig 3.19}
\end{figure}
\emph{\small Figure 3.19 Fundamental diagram 2020}


\begin{figure}
    \centering
    \includegraphics[scale=0.5]{september 2019 vs 2020 C601.png}
    \caption{}
    \label{Fig 3.20}
\end{figure}
\emph{\small Figure 3.20 Comparison September 2020 and September 2019}

## Number of Optimal Clusters, K

K-means  requires number of clusters K, as clustering parameter. Getting the optimal number of clusters is very significant in the analysis. If,for example, K is too high each time series starts representing a own cluster.

There is no a unique approach for finding the right number of clusters, I take in account a clustering quality measure, the soft-DTW similarity measure between nearest centroids and an empirical method to fix a minimum number of time series in each cluster.  

Since Soft-DTW is differentiable it could be also used as a function to evaluate the cohesion inside each cluster and the separation with respect to the nearest cluster. The silhouette coefficient is a measure of how similar a time series  is to its own cluster (cohesion) compared to other clusters (separation). The silhouette can be computed with the soft-DTW metric, it takes values in the range [-1, 1] \footnote{Rousseeuw: "Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis" pag 53-65}. 

Assume that the time series have been clustered via K-means.
For time series $t_i  \in C_k$ (time series $i$ in the cluster $C_k$ )
$a(t_i)$ is the mean distance between time series $t_i$ and all other time series in the same cluster:

$a(t_i)= \frac{1}{| C_k | -1} \sum_{j \in C_k} DTW^{\gamma}(t_i,t_j)$. 

$b(t_i)$ is defined as  the mean dissimilarity of the time series $t_i  \in C_k$ to  the nearest cluster $C_z$ ( where $C_k \neq C_z$) as the mean of the distance from the time series $t_i$ to all time series $\in C_z$:

$b(t_i) = \min_{k \neq z} \frac{1}{|C_z|}\sum_{j \in C_z} DTW^{\gamma}(t_i,t_j)$. 

Finally the silhouette coefficient for a time series is computed as follow: 

$s(t_i) = \frac{b(t_i)-a(t_i)}{max\{a(t_i),b(t_i)\}} \ \ if \ \ |C_k| >1$

The coefficients for every time series are averaged to have a global measure. The Mean Silhouette Coefficient for all time series is computed for the different number of clusters considered in the K-means algorithm to see how the number of clusters afflicts the analysis. 

I consider also the soft-DTW similarity measure between the nearest clusters, by taking in account their centroids and applying equation (2) :

- When $K=2$ becomes $DTW^{\gamma}(C_1,C_2)$ where $C_1$ and $C_2$ are centroids of the clusters.

- When $K=d$ where $d\geq 2$, $DTW^{\gamma}(C_j,C_z)$ is applied to all possible binary combinations of centroids forming a symmetric matrix of dimension $(d \times d)$ in which the minimum similarity between two centroids is selected.  

As the number of clusters $K$ increases the nearest clusters become closer each other until values of soft-DTW are stabilazed \ref{Fig 4.15}.

In addition as the number of clusters increases is more difficult to interpret the traffic behaviour captured by each cluster. To deal with this problem, following an empirical method \footnote{Zhang et.Al: "An empirical study to determine the optimal k in Ek-NNclus method" chapter 1}, I fixed a lower bound for the minimum number of observations in each cluster approximately at $\sqrt{ \frac{3}{2}*N}$. Where $N$ is the number of daily time series (365). If the number of time series within a cluster is lower than the fixed bound the cluster does not generalize well a particular traffic path, thus the interpretability of the cluster is too difficult.        


The code is implemented with the Python machine learning library for time series $\textbf{tslearn}$ \footnote{Tavernard et.Al: "Tslearn, A Machine Learning Toolkit for Time Series Data"}.  








# Appendix 

## Promenade Des Anglais out of order detectors

### 2019 

- C602 detector: 4.32 $%$ of missing values for both flow and occupancy rate. No data from 19/8 at 6 to 8:54. No data from 29/8 at 21:12 to 30/8 14:48. No data from 3/11 at 9 to 11:36. No data from 23/12 at 17:42 to 31/12.

- C077 detector: 56.72 $%$ of missing values. No data registered from 1/6 to 30/9 at 9:06. 

- C614: 7.05$%$ of missing values. No data available from 4/6 to 6/6 and from 21/8 to 3/9.

- C615: 46.81$%$ of missing values for the occupancy rate. No data registered from 1/6 to 30/9 at 9:06. 7.04 $%$ of missing values for the flow. No data registered from 4/6 at 17:00 to 6/6 at 15:00 and from 21/8 at 13:12 to 3/9 at 16:18 for the flow.

- C599: 12.68$%$ of missing values No data available from 12/7 to 15/7 and from 11/10 to 30/10.

- C598: 64.39$%$ of missing values for both flow and occupancy. 

### 2020

# References 

[1] Pedregosa, Varoquaux, Gramfort, Michel, Thirion. "Scikit-learn: Machine Learning in Python".

[2] Sardà-Espinosa. "Comparing Time-Series Clustering Algorithms in R Using the dtwclust Package".

[3] Sakoe, Chiba. “Dynamic programming algorithm optimization for spoken word recognition,” IEEE Transactions on Acoustics, Speech and Signal Processing.

[4] Cuturi, Blondel. "Soft-DTW: a Differentiable Loss Function for Time-Series". 

[5] Saeid Soheily-Khah. "Generalized k-means based clustering for temporal data under time warp".

[6] Rousseeuw. "Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis".

[7] Zhang, Bouadi, Martin. "An empirical study to determine the optimal k in Ek-NNclus method". 

[8] Tavenard, Faouzi, Vandewiele, Divo, Androz, Holtz, Payne, Yurchak. "Tslearn, A Machine Learning Toolkit for Time Series Data". 

[9] Treiber, Martin, Arne Kesting. "Traffic flow dynamics." 

[10] The code implemented is available at https://github.com/NicolaRonzoni/Multivariate-Time-series-clustering.git
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$
$$